In [ ]:
#load metadata & preprocessing

In [5]:
import pandas as pd 
import numpy as np 

df = pd.read_csv('../movie_lens_data/movies_metadata.csv')
df.head()

/opt/anaconda3/envs/root_2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
df.shape

(45466, 24)

In [7]:
df = df[df.id!='1997-08-20']
df = df[df.id!='2012-09-29']
df = df[df.id!='2014-01-01']
df['id'] = df['id'].astype(int)

In [9]:
links_map = pd.read_csv('../movie_lens_data/links_small.csv')
links_map.drop(columns=['imdbId'], inplace=True)

links_map = links_map[pd.notnull(links_map['tmdbId'])]
links_map['tmdbId'] = links_map['tmdbId'].astype(int)

links_map.columns = ['order_id', 'id']
links_map = links_map.merge(df[['title', 'id']], on='id').set_index('title')


In [10]:
new_df = df[df['id'].isin(links_map['id'])]
new_df.shape

(9099, 24)

In [12]:
#content-based recommender: credits, Genres, Keywords and Production Company Based Recommender

df_credits=pd.read_csv('../movie_lens_data/credits.csv')
df_credits['id'] = df_credits['id'].astype('int')
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [13]:
df_keywords=pd.read_csv('../movie_lens_data/keywords.csv')
df_keywords['id'] = df_keywords['id'].astype('int')
df_keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [14]:
new_df = new_df.merge(df_credits,on='id')
new_df = new_df.merge(df_keywords, on='id')
new_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [15]:
new_df = new_df[new_df['production_companies'].notnull()]

In [16]:
from ast import literal_eval
features = ['cast', 'crew', 'keywords', 'genres', 'production_companies']
for feature in features:
    new_df[feature] = new_df[feature].apply(literal_eval)

In [17]:
# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [18]:
# Lesser known actors and minor roles do not really affect people's opinion of a movie. Therefore we choose top 3 actors.
def get_top_three(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [19]:
new_df['director'] = new_df['crew'].apply(get_director)
features = ['cast', 'keywords', 'genres', 'production_companies']
for feature in features:
    new_df[feature] = new_df[feature].apply(get_top_three)

In [20]:
new_df[['title', 'cast', 'director', 'keywords', 'genres', 'production_companies']].head()

,title,cast,director,keywords,genres,production_companies
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]",[Pixar Animation Studios]
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope Co..."
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]","[Warner Bros., Lancaster Gate]"
3,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devine]",Forest Whitaker,"[based on novel, interracial relationship, sin...","[Comedy, Drama, Romance]",[Twentieth Century Fox Film Corporation]
4,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short]",Charles Shyer,"[baby, midlife crisis, confidence]",[Comedy],"[Sandollar Productions, Touchstone Pictures]"


In [21]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [22]:
features = ['cast', 'keywords', 'director', 'genres', 'production_companies']

for feature in features:
    new_df[feature] = new_df[feature].apply(clean_data)

In [23]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres']) + ' ' + ' '.join(x['production_companies'])

new_df['soup'] = new_df.apply(create_soup, axis=1)

In [24]:
new_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,cast,crew,keywords,director,soup
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[animation, comedy, family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,NaN,Toy Story,False,7.7,5415.0,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy]",johnlasseter,jealousy toy boy tomhanks timallen donrickles ...
1,False,NaN,65000000,"[adventure, fantasy, family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[boardgame, disappearance, basedonchildren'sbook]",joejohnston,boardgame disappearance basedonchildren'sbook ...
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[romance, comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, bestfriend, duringcreditsstinger]",howarddeutch,fishing bestfriend duringcreditsstinger walter...
3,False,NaN,16000000,"[comedy, drama, romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[basedonnovel, interracialrelationship, single...",forestwhitaker,basedonnovel interracialrelationship singlemot...
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[baby, midlifecrisis, confidence]",charlesshyer,baby midlifecrisis confidence stevemartin dian...


In [25]:
#collaborative filtering

In [122]:
from surprise import Reader
reader = Reader()
ratings_df = pd.read_csv('../movie_lens_data/ratings_small.csv')
ratings_df.drop(columns=['timestamp'], inplace=True)
ratings_df.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [123]:
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split

reader = Reader()
ratings = Dataset.load_from_df(ratings_df, reader)

train_ratings, test_ratings = train_test_split(ratings, test_size=0.2, random_state = 42)
print("Size of trainset: ", train_ratings.n_ratings)
print("Size of testset: ", len(test_ratings))

Size of trainset:  80003
Size of testset:  20001


In [124]:
from surprise import SVD
svd = SVD()
svd.fit(train_ratings)

from surprise import accuracy
train_pred_svd = svd.test(train_ratings.build_testset())
test_pred_svd = svd.test(test_ratings)
print("RMSE on training data (SVD): ", accuracy.rmse(train_pred_svd, verbose = False))
print("RMSE on test data (SVD): ", accuracy.rmse(test_pred_svd, verbose = False))

RMSE on training data (SVD):  0.6422406961238438
RMSE on test data (SVD):  0.9029356843747463


In [125]:
indices_map = links_map.set_index('id')
new_df = new_df.reset_index()
titles = new_df['title']
indices = pd.Series(new_df.index, index=new_df['title'])

In [126]:
def hybrid_model(userId, title, cosine_sim):
    idx = indices[title]
    tmdbId = links_map.loc[title]['id']
    movie_id = links_map.loc[title]['order_id']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:30]
    
    movie_indices = [i[0] for i in sim_scores]
    movies = new_df.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['order_id']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [127]:
#collaborative + lists of features

In [128]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(new_df['soup'])
count_matrix.shape

(9219, 24507)

In [129]:
from sklearn.metrics.pairwise import linear_kernel
feature_cosine_sim = linear_kernel(count_matrix, count_matrix)

In [26]:
hybrid_model(1, 'The Dark Knight Rises', feature_cosine_sim)

,title,vote_count,vote_average,id,est
6623,The Prestige,4510.0,8.0,1124,3.554846
6981,The Dark Knight,12269.0,8.3,155,3.346354
252,Murder in the First,140.0,6.9,8438,3.153880
6218,Batman Begins,7511.0,7.5,272,3.137553
5,Heat,1886.0,7.7,949,3.097724
5521,Angels with Dirty Faces,74.0,7.6,13696,3.031414
5098,The Enforcer,21.0,7.4,26712,2.958927
467,Romeo Is Bleeding,36.0,5.7,2088,2.863429
5008,The Public Enemy,77.0,7.3,17687,2.839679
2599,Batman: Mask of the Phantasm,218.0,7.4,14919,2.835538


In [27]:
hybrid_model(5, 'The Dark Knight Rises', feature_cosine_sim)

,title,vote_count,vote_average,id,est
6981,The Dark Knight,12269.0,8.3,155,4.535429
252,Murder in the First,140.0,6.9,8438,4.271315
6623,The Prestige,4510.0,8.0,1124,4.252130
5,Heat,1886.0,7.7,949,4.167692
6218,Batman Begins,7511.0,7.5,272,4.165172
467,Romeo Is Bleeding,36.0,5.7,2088,4.036345
2599,Batman: Mask of the Phantasm,218.0,7.4,14919,4.015450
5521,Angels with Dirty Faces,74.0,7.6,13696,3.963437
5098,The Enforcer,21.0,7.4,26712,3.924228
4387,Sharky's Machine,31.0,6.1,14664,3.885791


In [39]:
hybrid_model(1, 'Toy Story', feature_cosine_sim)

,title,vote_count,vote_average,id,est
627,A Close Shave,223.0,7.5,532,3.672137
7629,Toy Story 3,4710.0,7.6,10193,3.555164
933,The Wrong Trousers,266.0,7.6,531,3.534359
2751,Creature Comforts,29.0,7.3,54825,3.344096
543,The Aristocats,1287.0,7.1,10112,3.248487
6695,Ratatouille,4510.0,7.5,2062,3.219947
7307,Up,7048.0,7.8,14160,3.156952
2228,The Iron Giant,1470.0,7.6,10386,3.119030
3833,"Monsters, Inc.",6150.0,7.5,585,3.068660
2522,Toy Story 2,3914.0,7.3,863,3.039475


In [40]:
hybrid_model(5, 'Toy Story', feature_cosine_sim)

,title,vote_count,vote_average,id,est
627,A Close Shave,223.0,7.5,532,4.508362
7307,Up,7048.0,7.8,14160,4.506887
7629,Toy Story 3,4710.0,7.6,10193,4.499642
933,The Wrong Trousers,266.0,7.6,531,4.457327
6695,Ratatouille,4510.0,7.5,2062,4.367012
521,Aladdin,3495.0,7.4,812,4.171721
1883,A Bug's Life,2379.0,6.8,9487,4.168428
2228,The Iron Giant,1470.0,7.6,10386,4.149452
2751,Creature Comforts,29.0,7.3,54825,4.036890
3833,"Monsters, Inc.",6150.0,7.5,585,4.016414


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
def overview_tfidf(dataset):
    curr = dataset
    tfidf = TfidfVectorizer(stop_words='english')

    new_df['overview'] = curr['overview'].fillna('')
    tfidf_matrix = tfidf.fit_transform(curr['overview'])
    return tfidf_matrix

In [29]:
overview_tfidf_matrix = overview_tfidf(new_df)
overview_tfidf_matrix.shape

(9219, 29727)

In [30]:
def hybrid_model(userId, title, cosine_sim):
    idx = indices[title]
    tmdbId = links_map.loc[title]['id']
    movie_id = links_map.loc[title]['order_id']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:30]
    
    movie_indices = [i[0] for i in sim_scores]
    movies = new_df.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['order_id']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [31]:
from sklearn.metrics.pairwise import linear_kernel
overview_cosine_sim = linear_kernel(overview_tfidf_matrix, overview_tfidf_matrix)

In [34]:
hybrid_model(1, 'The Dark Knight Rises', overview_cosine_sim)

,title,vote_count,vote_average,id,est
6981,The Dark Knight,12269.0,8.3,155,3.346354
6218,Batman Begins,7511.0,7.5,272,3.137553
2716,JFK,513.0,7.5,820,3.030749
8334,"Batman: The Dark Knight Returns, Part 2",426.0,7.9,142061,2.974755
7330,The File on Thelma Jordon,8.0,6.3,35404,2.955986
8791,The Young Savages,7.0,5.5,86616,2.930802
8265,"Batman: The Dark Knight Returns, Part 1",410.0,7.7,123025,2.930022
6748,Fracture,908.0,7.1,6145,2.905538
2599,Batman: Mask of the Phantasm,218.0,7.4,14919,2.835538
4544,Q & A,22.0,6.6,31598,2.811690


In [35]:
hybrid_model(5, 'The Dark Knight Rises', overview_cosine_sim)

,title,vote_count,vote_average,id,est
6981,The Dark Knight,12269.0,8.3,155,4.535429
6218,Batman Begins,7511.0,7.5,272,4.165172
8265,"Batman: The Dark Knight Returns, Part 1",410.0,7.7,123025,4.076198
6748,Fracture,908.0,7.1,6145,4.051374
2599,Batman: Mask of the Phantasm,218.0,7.4,14919,4.015450
8791,The Young Savages,7.0,5.5,86616,3.998265
2716,JFK,513.0,7.5,820,3.971565
8334,"Batman: The Dark Knight Returns, Part 2",426.0,7.9,142061,3.936510
7439,Law Abiding Citizen,1522.0,7.2,22803,3.863959
7330,The File on Thelma Jordon,8.0,6.3,35404,3.854967


In [37]:
hybrid_model(1, 'Toy Story', overview_cosine_sim)

,title,vote_count,vote_average,id,est
7629,Toy Story 3,4710.0,7.6,10193,3.555164
6267,The 40 Year Old Virgin,2020.0,6.2,6957,3.286962
910,Rebel Without a Cause,351.0,7.6,221,3.227337
2522,Toy Story 2,3914.0,7.3,863,3.039475
6364,Match Point,1134.0,7.2,116,3.034528
643,I Shot Andy Warhol,32.0,6.1,26890,3.014376
2219,Radio Days,131.0,7.0,30890,3.008445
2710,Bound for Glory,22.0,7.0,42232,2.940634
4757,"What's Up, Tiger Lily?",49.0,5.6,21449,2.890900
6708,Factory Girl,83.0,6.2,12271,2.882764


In [38]:
hybrid_model(5, 'Toy Story', overview_cosine_sim)

,title,vote_count,vote_average,id,est
7629,Toy Story 3,4710.0,7.6,10193,4.499642
910,Rebel Without a Cause,351.0,7.6,221,4.329941
643,I Shot Andy Warhol,32.0,6.1,26890,4.126217
6267,The 40 Year Old Virgin,2020.0,6.2,6957,3.994463
5043,Rivers and Tides,13.0,7.3,30140,3.958209
2522,Toy Story 2,3914.0,7.3,863,3.946270
6364,Match Point,1134.0,7.2,116,3.936979
4757,"What's Up, Tiger Lily?",49.0,5.6,21449,3.927671
2443,The Funhouse,69.0,5.8,13555,3.877425
2567,Man on the Moon,444.0,6.9,1850,3.842248


# Interactive interface for recommendation on a new user
How does it work:
1. use IMDB formula to find top 250 movies -> popular movies that people are  more likely watched before
2. randomly pick 20 movies from top 250 list and let user input ratings
3. use hybrid model from previous to recommend movies for this new user, based on the ratings this user gives on these 20 samples. 

In [ ]:
# weighted rating (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C
# R = average for the movie (mean) = (Rating)
# v = number of votes for the movie = (votes)
# m = minimum votes required to be listed in the Top 250 (currently 25,000)
# C = the mean vote across the whole report

In [38]:
C = new_df['vote_average'].mean()
C

6.354452760603103

In [70]:
m = 25000
new_df['weighted_rating'] = (new_df['vote_count'] / (new_df['vote_count']+m)) * new_df['vote_average'] + (m / (new_df['vote_count']+m)) * C


top_250_df = new_df.sort_values('weighted_rating',  ascending = False).head(250)




In [ ]:

def interactive_user_input():
    result = []
    random_20 = top_250_df.sample(20)
    count = 20
    print("Please rate following 20 movies, input a rating between 0-5 (increment by 0.5).")
    print("If you did not know this movie, simply hit enter to skip.\n")
    for index, row in random_20.iterrows():
        print("title: ", row['title'])
        print("Overview: ", row['overview'])
        print("Release date: ", row['release_date'])
        print("\n")
        print("Rating: ")
        rating = input()
        count-=1;
        if(count > 0):
            print("\n")
            print(count, "more ...")
        else:
            print("finished!")
        result.append([row['id'], rating, row['title']])
    return result

result = interactive_user_input()

In [ ]:
new_ratings = ratings_df
for item in result:
    if (item[1] != ''):
        new_ratings_row = pd.DataFrame([[674,item[0],item[1]]], columns=['userId','movieId','rating'])
        new_ratings = pd.concat([new_ratings, new_ratings_row])
fav_movies = sorted(result,key=lambda l:l[1], reverse=True)

In [ ]:
# take top 2 this user's favorte movie and output top 10 recommendations 
top_1 = hybrid_model(674, fav_movies[0][2], feature_cosine_sim)
top_2 = hybrid_model(674, fav_movies[1][2], feature_cosine_sim)
final = pd.concat([top_1, top_2])
final.sort_values('est', ascending = False).head(10)